# Track the redun run

LaminDB only tracks the inputs, outputs and the execution ID of any redun run.

More finegrained information is tracked by redun.

In [ ]:
import lamindb as ln
import os
import json

ln.track()

## Query input data and create a run

This is an artificial situation in which we pretend our input data comes from a notebook with ID `0ymQDuqM5Lwq`:

In [ ]:
inputs = ln.File.select(transform__stem_id="0ymQDuqM5Lwq")

inputs.df()

Let's create a LaminDB run record and link it agains the input files:

In [ ]:
inputs

Let's create a run:

In [ ]:
transform = ln.Transform.select(name="lamin-redun-fasta").one()  # load a pipeline
run = ln.Run(
    transform=transform
)  # create a run record that is linked against the pipeline
run.save()
run

And register the inputs for the run:

In [ ]:
run.input_files.set(inputs)

We can also see it here:

In [ ]:
inputs[0].input_of.all()

## Execute redun

In order to pass it to the redun CLI, export the `run.id` as an env variable:

In [ ]:
os.environ["LAMINDB_RUN_ID"] = run.id

Let us now call the workflow (tagging with the `run.id` is optional but ensures convenient query via `redun log` later):

In [ ]:
!redun run workflow.py main --run-id $LAMINDB_RUN_ID --tag run-id=$LAMINDB_RUN_ID  1> redun_stdout.txt 2>redun_stderr.txt

Inspect the output:

In [ ]:
!cat redun_stdout.txt

In [ ]:
!tail -1 redun_stderr.txt

In [ ]:
!redun log --exec --exec-tag run-id=$LAMINDB_RUN_ID --format json --no-pager > redun_exec.json

In [ ]:
cat redun_exec.json

In [ ]:
redun_exec = json.load(open("redun_exec.json"))

redun_exec

## Track redun outputs and execution ID

In [ ]:
run = ln.Run.select(id=run.id).one()
run.external_id = redun_exec["id"]
run.save()

There is just a single output file to track, here:

In [ ]:
ln.File(data="data/results.tgz", run=run).save()

## View the database content

In [ ]:
ln.view()